This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify curves of genus 7 with an irrational $g^2_6$ which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 1 hour for completion.

In [1]:
load("preamble.sage")

Construct the sets of $\mathbb{F}_{2^i}$-rational points of the quadratic twist $X$ of $\mathbb{P}^2 \times \mathbb{P}^2$ for $i=1,2,3$.

In [2]:
F = GF(2)
P.<x0,x1,x2,y0,y1,y2> = PolynomialRing(F, 6)

In [3]:
F4.<a> = GF(4)
S14 = [vector(t) for t in ProjectiveSpace(F4, 2).rational_points()]
S = [(x, vector(F4, tuple(i.conjugate() for i in x))) for x in S14]
for v in S:
    for x in v:
        x.set_immutable()
len(S)

21

In [4]:
F16.<a2> = F4.extension(2)
S4 = list(itertools.product(S14, repeat=2))
len(S4)

441

In [5]:
F64.<a3> = F4.extension(3)
S164 = [vector(t) for t in ProjectiveSpace(F64, 2).rational_points()]
S8 = [(x, vector(F64, tuple(i.conjugate() for i in x))) for x in S164]
len(S8)

4161

Construct the group $\mathrm{GL}(3,\mathbb{F}_4)$, presented as a subgroup of $\mathrm{GL}(6, \mathbb{F}_4)$.

In [6]:
l0 = [block_matrix(2,2,[g.matrix(), 0, 0, g.matrix().conjugate()], subdivide=False) for g in GL(3,F4).gens()]
G0 = GL(6,F4).subgroup(l0)
G0.order()

181440

Use an orbit lookup tree to find $\mathrm{GL}(3,\mathbb{F}_4)$-orbit representatives for 7-tuples of $\mathbb{F}_2$-points in $X$.

In [7]:
def apply_group_elem(g, x, S=S, a=a):
    g1 = g.submatrix(nrows=3,ncols=3)
    g2 = g.submatrix(row=3,col=3)
    v1 = g1*x[0]
    v2 = g2*x[1]
    for i in range(3):
        if (v1, v2) in S:
            v1.set_immutable()
            v2.set_immutable()
            return (v1, v2)
        v1 = a*v1
        v2 = (a+1)*v2
    raise ValueError

In [8]:
amat = diagonal_matrix(F4, [a,a,a,a+1,a+1,a+1])

In [9]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F4, x[0]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,g.matrix().conjugate()], subdivide=False) for g in G1.gens()]
    return GL(6, F4).subgroup(l0)

In [10]:
def optimized_rep(g):
    return g.matrix()

In [11]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}
tree = build_orbit_tree(G0, S, 7, methods, verbose=False)

In [12]:
nodes = {6: green_nodes(tree, 6), 7: green_nodes(tree, 7)}
len(nodes[6]), len(nodes[7])

(8, 10)

For each orbit representative, for each 2-dimensional space of $(1,1)$-hypersurfaces passing through those points, solve for $(2,2)$-hypersurfaces passing through exactly those $\mathbb{F}_2$-rational points, and impose the desired point counts over $\mathbb{F}_4$ and $\mathbb{F}_8$.

In [13]:
monos11 = [x0*y0, x1*y1, x2*y2,
           x1*y0 + x0*y1, a*x1*y0 + (a+1)*x0*y1,
           x2*y0 + x0*y2, a*x2*y0 + (a+1)*x0*y2,
           x1*y2 + x2*y1, a*x1*y2 + (a+1)*x2*y1]
monos2a = [prod(i) for i in itertools.product([prod(j) for j in itertools.combinations_with_replacement([x0,x1,x2],2)], 
                                              [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2],2)])]
monos2b = [prod(t) for t in itertools.product(monos11, repeat=2)]
tmp = [vector(F4, (x.coefficient(mu.exponents()[0]) for mu in monos2a)) for x in monos2b]
tmp2 = []
monos2 = []
for i in range(len(monos2b)):
    if Matrix(F4, tmp2 + [tmp[i]]).rank() > Matrix(F4, tmp2).rank():
        tmp2.append(tmp[i])
        monos2.append(monos2b[i])
len(monos2)

36

In [14]:
coords1 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos11)) for x in S}
coords2 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos2)) for x in S}

In [15]:
def redundancy(gens, F=F, F4=F4, monos11=monos11, monos2a=monos2a, tmp2=tmp2):
    ans = []
    for m in monos11:
        for v in gens:
            tmp3 = vector(F4, ((v*m).coefficient(mu.exponents()[0]) for mu in monos2a))
            w = Matrix(tmp2 + [tmp3]).left_kernel().gens()[0][:-1]
            ans.append(vector(F, (w[i] for i in range(len(w)))))
    return ans

In [ ]:
curves = defaultdict(list)
for s1 in [6, 7]:
    for vecs in nodes[s1]:
        V = Matrix([coords1[x] for x in vecs]).right_kernel()
        for W in V.subspaces(2):
            (v1, v2) = W.basis()
            gens1 = sum(v1[i]*monos11[i] for i in range(9))
            gens2 = sum(v2[i]*monos11[i] for i in range(9))
            pts = [x for x in S if gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
            perp = Matrix([coords2[x] for x in pts])
            target = vector(F, (0 if x in vecs else 1 for x in pts))
            pts2 = 0
            pts3 = 0
            for w in solve_right_iterator(perp, target, redundancy, [gens1, gens2]):
                gens3 = sum(w[i]*monos2[i] for i in range(36))
                if pts2 == 0:
                    pts2 = [x for x in S4 if gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
                s2 = sum(1 for x in pts2 if gens3(*x[0], *x[1]) == 0)
                if (s1 == 6 and s2 == 18) or (s1 == 7 and s2 == 15):
                    if pts3 == 0:
                        pts3 = [x for x in S8 if gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
                    s3 = sum(1 for x in pts3 if gens3(*x[0], *x[1]) == 0)
                    if (s1 == 6 and s3 == 12) or (s1 == 7 and s3 == 7):
                        curves[(s1,s2,s3)].append([gens1, gens2, gens3])

In [ ]:
[(s, len(curves[s])) for s in curves]

Enforce the desired point count conditions over $\mathbb{F}_{2^i}$ for $i=4,5$, yielding no results.

In [ ]:
count_gens = {}
for n in range(1, 6):
    if n%2 == 1:
        count_gens[n] = [x0^(2^n)+y0, x1^(2^n)+y1, x2^(2^n)+y2, y0^(2^n)+x0, y1^(2^n)+x1, y2^(2^n)+x2]
    else:
        count_gens[n] = [x^(2^n)+x for x in P.gens()]

def count_by_ideal(gens, n):
    J = P.change_ring(F4).ideal(gens + count_gens[n])
    if n%2 == 1:
        return (J.vector_space_dimension() - 1) // (2^(2*n)-1)
    else:
        return (J.vector_space_dimension() - 2^(3*n+1) + 1) // (2^n-1)^2

In [ ]:
for n in range(4, 6):
    tmp = [t[:n] for t in targets7]
    tmp2 = [s for s in curves if len(s) == n-1]    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
    print([(s, len(curves[s])) for s in curves if len(s) == n])

In [ ]:
assert all(len(s) < 5 for s in curves)

Close out this case.

In [ ]:
closeout()